In [1]:
from functools import reduce
import csbgnpy.pd.io.sbgnml
import sbgn2an.stories
import sbgn2an.ini
import pypint
import multiprocessing

This notebook has been executed using the docker image `pauleve/clockcycle:v0`

## Model loading

From [1 - Merge CYCLE and CLOCK (short)](1%20-%20Merge%20CYCLE%20and%20CLOCK%20(short).ipynb)

In [2]:
merge_file = "generated/MERGE.sbgnml"
merge = csbgnpy.pd.io.sbgnml.read(merge_file)

In [3]:
clock = csbgnpy.pd.io.sbgnml.read("generated/CLOCK.sbgnml")
rbe2f = csbgnpy.pd.io.sbgnml.read("generated/CYCLE.sbgnml")
clock_ids = set([e.id for e in clock.entities]) # + [p.id for p in clock.processes])
rbe2f_ids = set([e.id for e in rbe2f.entities]) # + [p.id for p in rbe2f.processes])
clock_only = clock_ids - rbe2f_ids
rbe2f_only = rbe2f_ids - clock_ids

From [2 - Markers.ipynb](2%20-%20Markers.ipynb):

In [4]:
%store -r cycle_markers
%store -r clock_markers

### Initial state

In [5]:
inis = sbgn2an.ini.get_ini(merge, in_ini = [merge.get_entity("Complex([SubMacromolecule([][@|@|@|@]pRB*)|SubMacromolecule([][@]DP1*)|SubMacromolecule([][@Ser364|@Ser31|@Ser403|@Lys|@]E2F1)][][]pRB*/E2F1/DP1*#Compartment(nucleoplasm))", by_string = True)])#, stories = stories)
ini = inis[0]

### Conversion to internal identifiers

In [6]:
ini = [e.id for e in ini]

In [7]:
def get_map(a):
    if a in clock_only:
        return "CLOCK"
    if a in rbe2f_only:
        return "RBE2F"
    return "MERGE"
def pretty_node(a):
    if a.startswith("epn_"):
        e = merge.get_entity(a, by_id=True)
    else:
        e = merge.get_process(a, by_id=True)
    return "{}::{}".format(get_map(a), str(e))

In [8]:
an = pypint.load(merge_file, initial_state=ini)

In [9]:
an_rbe2f_markers = dict([(p, dict([(merge.get_entity(m, by_string=True).id,1) for m in ms])) \
                         for p,ms in cycle_markers.items()])
an_clock_markers = dict([(p, dict([(merge.get_entity(m, by_string=True).id,1) for m in ms])) \
                         for p,ms in clock_markers.items()])

In [10]:
def pretty_mutations(m):
    return dict([(pretty_node(a), i) for (a,i) in m.items()])

In [11]:
def list_interesting(ms, interest):
    i2m = {}
    for mi, m in enumerate(ms):
        li = [(a,i) for (a,i) in m.items() if a in interest]
        arity = len(m)
        for (a,i) in li:
            a = pretty_node(a)
            if (a,i) not in i2m:
                i2m[(a,i)] = set()
            i2m[(a,i)].add(arity)
            #if arity not in i2m[(a,i)]:
            #    i2m[(a,i)][arity] = []
            #i2m[(a,i)][arity].append(mi)
    si2m = {}
    for k, v in sorted(i2m.items()):
        si2m[k] = v
    return si2m

## RB/E2F markers

In [12]:
ms = an.cutsets(an_rbe2f_markers["G0"], maxsize=5)
#[pretty_mutations(m) for m in ms if clock_ids.intersection(m)]
list_interesting(ms, clock_ids)

This computation is an *under-approximation*: returned cut-sets are all valid, but they may be non-minimal, and some cut-sets may be missed.

Limiting results to cut-sets with at most 5 elements. Use `maxsize` argument to change.

{('MERGE::Macromolecule([][]HSP90#Compartment([]cytoplasm))', 1): {3, 4}}

In [13]:
ms = an.cutsets(an_rbe2f_markers["earlyG1"], maxsize=5)
#[pretty_mutations(m) for m in ms if clock_ids.intersection(m)]
list_interesting(ms, clock_ids)

This computation is an *under-approximation*: returned cut-sets are all valid, but they may be non-minimal, and some cut-sets may be missed.

Limiting results to cut-sets with at most 5 elements. Use `maxsize` argument to change.

{('MERGE::Macromolecule([][]HSP90#Compartment([]cytoplasm))', 1): {1}}

In [14]:
ms = an.cutsets(an_rbe2f_markers["lateG1"], maxsize=5)
list_interesting(ms, clock_ids)

This computation is an *under-approximation*: returned cut-sets are all valid, but they may be non-minimal, and some cut-sets may be missed.

Limiting results to cut-sets with at most 5 elements. Use `maxsize` argument to change.

{('MERGE::Macromolecule([][]HSP90#Compartment([]cytoplasm))', 1): {2, 3}}

In [15]:
ms = an.cutsets(an_rbe2f_markers["earlyS"], maxsize=5)
list_interesting(ms, clock_ids)

This computation is an *under-approximation*: returned cut-sets are all valid, but they may be non-minimal, and some cut-sets may be missed.

Limiting results to cut-sets with at most 5 elements. Use `maxsize` argument to change.

{}

In [16]:
ms = an.cutsets(an_rbe2f_markers["lateS"], maxsize=5)
list_interesting(ms, clock_ids)

This computation is an *under-approximation*: returned cut-sets are all valid, but they may be non-minimal, and some cut-sets may be missed.

Limiting results to cut-sets with at most 5 elements. Use `maxsize` argument to change.

{('MERGE::Macromolecule([][]HSP90#Compartment([]cytoplasm))', 1): {3, 4, 5}}

In [17]:
ms = an.cutsets(an_rbe2f_markers["G2"], maxsize=5, exclude_initial_state=True)
list_interesting(ms, clock_ids)

This computation is an *under-approximation*: returned cut-sets are all valid, but they may be non-minimal, and some cut-sets may be missed.

Limiting results to cut-sets with at most 5 elements. Use `maxsize` argument to change.

{}

In [18]:
ms = an.cutsets(an_rbe2f_markers["M"], maxsize=5, exclude_initial_state=True)
list_interesting(ms, clock_ids)

This computation is an *under-approximation*: returned cut-sets are all valid, but they may be non-minimal, and some cut-sets may be missed.

Limiting results to cut-sets with at most 5 elements. Use `maxsize` argument to change.

{}

## CLOCK markers

In [19]:
an_clock_markers.keys()

dict_keys(['RORG', 'SIRT1', 'BMAL1-CLOCK', 'PER12-CRY12'])

In [20]:
ms = an.cutsets(an_clock_markers["RORG"], maxsize=5, exclude_initial_state=True)
list_interesting(ms, rbe2f_ids)

This computation is an *under-approximation*: returned cut-sets are all valid, but they may be non-minimal, and some cut-sets may be missed.

Limiting results to cut-sets with at most 5 elements. Use `maxsize` argument to change.

{}

In [21]:
ms = an.cutsets(an_clock_markers["RORG"], maxsize=5)
list_interesting(ms, rbe2f_ids)

This computation is an *under-approximation*: returned cut-sets are all valid, but they may be non-minimal, and some cut-sets may be missed.

Limiting results to cut-sets with at most 5 elements. Use `maxsize` argument to change.

{}

In [22]:
ms = an.cutsets(an_clock_markers["SIRT1"], maxsize=5)
list_interesting(ms, rbe2f_ids)

This computation is an *under-approximation*: returned cut-sets are all valid, but they may be non-minimal, and some cut-sets may be missed.

Limiting results to cut-sets with at most 5 elements. Use `maxsize` argument to change.

{}

In [23]:
ms = an.cutsets(an_clock_markers["BMAL1-CLOCK"], maxsize=5)
list_interesting(ms, rbe2f_ids)

This computation is an *under-approximation*: returned cut-sets are all valid, but they may be non-minimal, and some cut-sets may be missed.

Limiting results to cut-sets with at most 5 elements. Use `maxsize` argument to change.

{}

In [24]:
ms = an.cutsets(an_clock_markers["PER12-CRY12"], maxsize=5, exclude_initial_state=True)
list_interesting(ms, rbe2f_ids)

This computation is an *under-approximation*: returned cut-sets are all valid, but they may be non-minimal, and some cut-sets may be missed.

Limiting results to cut-sets with at most 5 elements. Use `maxsize` argument to change.

{}

In [25]:
ms = an.cutsets(an_clock_markers["PER12-CRY12"], maxsize=5, exclude_initial_state=False)
list_interesting(ms, rbe2f_ids)

This computation is an *under-approximation*: returned cut-sets are all valid, but they may be non-minimal, and some cut-sets may be missed.

Limiting results to cut-sets with at most 5 elements. Use `maxsize` argument to change.

{}